In [1]:
import pandas as pd
import random
import os
import osmnx as ox

/home/jptalusan/anaconda3/envs/py39/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
SEED = 100
OUTPUT_DIR = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data'

In [3]:
random.seed(SEED)

load_dist_path = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/loads_by_scheduled_time.pkl'
df = pd.read_pickle(load_dist_path)

df['sample_load'] = df['load'].apply(lambda x: random.choice(x))
df = df.reset_index()
df.drop('load', axis=1, inplace=True)
display(df.head())
out_fp = os.path.join(OUTPUT_DIR, 'sampled_load.pkl')
df.to_pickle(out_fp)

,route_id_direction,block_abbr,stop_id_original,time,IsWeekend,sample_load
0,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:13,0,2.0
1,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:13,1,3.0
2,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:41,0,7.0
3,14_FROM DOWNTOWN,1400,1SJAMNM,06:21:13,0,10.0
4,14_FROM DOWNTOWN,1400,1SJAMNM,06:21:13,1,4.0


# Boardings
* I'll be using boardings for the simulation and giving it an affine transformation that is very much empirical/random without any thought into it.
* $ons_{new} = \lfloor 1.5*ons_{old} + 1 \rfloor$

In [10]:
import numpy as np
random.seed(SEED)

ons_dist_path = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/ons_by_scheduled_time.pkl'
df = pd.read_pickle(ons_dist_path)

df['sample_ons'] = df['ons'].apply(lambda x: random.choice(x))
df = df.reset_index()
df.drop('ons', axis=1, inplace=True)

df['sample_ons'] = df['sample_ons'] * 1.5 + 1
df['sample_ons'] = df['sample_ons'].apply(np.floor)

display(df.head())
out_fp = os.path.join(OUTPUT_DIR, 'sampled_ons.pkl')
df.to_pickle(out_fp)

,route_id_direction,block_abbr,stop_id_original,time,IsWeekend,sample_ons
0,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:13,0,1.0
1,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:13,1,1.0
2,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:41,0,1.0
3,14_FROM DOWNTOWN,1400,1SJAMNM,06:21:13,0,1.0
4,14_FROM DOWNTOWN,1400,1SJAMNM,06:21:13,1,1.0


# Alighting
* $offs_{new} = \lfloor 1.5*offs_{old} + 1 \rfloor$

In [15]:
import numpy as np
random.seed(SEED)

offs_dist_path = '../scenarios/baseline/data/offs_by_scheduled_time.pkl'
df = pd.read_pickle(offs_dist_path)

df['sample_offs'] = df['offs'].apply(lambda x: random.choice(x))
df = df.reset_index()
df.drop('offs', axis=1, inplace=True)

df['sample_offs'] = df['sample_offs'] * 1.5 + 1
df['sample_offs'] = df['sample_offs'].apply(np.floor)

display(df.head())
out_fp = os.path.join(OUTPUT_DIR, 'sampled_offs.pkl')
df.to_pickle(out_fp)

,route_id_direction,block_abbr,stop_id_original,time,IsWeekend,sample_offs
0,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:13,0,1.0
1,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:13,1,1.0
2,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:41,0,1.0
3,14_FROM DOWNTOWN,1400,1SJAMNM,06:21:13,0,1.0
4,14_FROM DOWNTOWN,1400,1SJAMNM,06:21:13,1,1.0


# Travel time

In [7]:
random.seed(SEED)

tt_dist_path = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/travel_time_by_scheduled_time.pkl'
df = pd.read_pickle(tt_dist_path)

df['time_to_next_stop'] = df['time_to_next_stop'].apply(lambda x: [item for item in x if item >= 0])
df['has_positive'] = df['time_to_next_stop'].apply(lambda x: len([item for item in x if item >= 0]) > 0)
df = df[df['has_positive'] == True]
df['sample_time_to_next_stop'] = df['time_to_next_stop'].apply(lambda x: random.choice(x))
df = df.reset_index()
df.drop(['time_to_next_stop', 'has_positive'], axis=1, inplace=True)

out_fp = os.path.join(OUTPUT_DIR, 'sampled_travel_times.pkl')
df.to_pickle(out_fp)

In [8]:
import datetime as dt
tdf = df.reset_index().query("route_id_direction == '14_FROM DOWNTOWN' and block_abbr == 1400 and stop_id_original == 'MCC4_20'")
tdf['time'] = pd.to_datetime(tdf['time'], format='%H:%M:%S')
tdf[tdf['time'] == '1900-01-01 14:15:00	']

,index,route_id_direction,block_abbr,stop_sequence,stop_id_original,time,IsWeekend,sample_time_to_next_stop
18,18,14_FROM DOWNTOWN,1400,1,MCC4_20,1900-01-01 14:15:00,0,872.0
19,19,14_FROM DOWNTOWN,1400,1,MCC4_20,1900-01-01 14:15:00,1,855.0


In [9]:
distance_path = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/sampled_travel_times.pkl'
df = pd.read_pickle(distance_path)
df

,route_id_direction,block_abbr,stop_sequence,stop_id_original,time,IsWeekend,sample_time_to_next_stop
0,14_FROM DOWNTOWN,1400,1,MCC4_20,06:15:00,0,498.0
1,14_FROM DOWNTOWN,1400,1,MCC4_20,06:15:00,1,870.0
2,14_FROM DOWNTOWN,1400,1,MCC4_20,06:18:00,0,212.0
3,14_FROM DOWNTOWN,1400,1,MCC4_20,06:18:00,1,584.0
4,14_FROM DOWNTOWN,1400,1,MCC4_20,07:15:00,0,936.0
...,...,...,...,...,...,...,...
533811,9_TO DOWNTOWN,8600,6,2AVJUNSF,07:48:54,0,28.0
533812,9_TO DOWNTOWN,8600,6,2AVJUNSF,07:48:57,0,34.0
533813,9_TO DOWNTOWN,8600,7,2ASTOSM,07:50:02,0,266.0
533814,9_TO DOWNTOWN,8600,7,2ASTOSM,07:50:06,0,328.0


In [8]:
tdf = df.reset_index().query("route_id_direction == '86_FROM NASHVILLE' and block_abbr == 8602 and stop_id_original == 'MCC4_23'")
tdf

,index,route_id_direction,block_abbr,stop_sequence,stop_id_original,time,IsWeekend,sample_time_to_next_stop
513521,513521,86_FROM NASHVILLE,8602,1,MCC4_23,15:48:00,0,412.0


In [13]:
import datetime as dt
a = dt.datetime.strptime("2021-08-23 16:49:00", "%Y-%m-%d %H:%M:%S")
b = dt.datetime.strptime("2021-08-23 17:49:00", "%Y-%m-%d %H:%M:%S")
c = dt.datetime.strptime("2021-08-23 12:00:00", "%Y-%m-%d %H:%M:%S")
d = c + (b-a)
d

datetime.datetime(2021, 8, 23, 13, 0)

In [6]:
fp = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/stops_node_matching.pkl'
df = pd.read_pickle(fp)
cn = df.query("stop_id_original == 'MCC5_1'")['nearest_node'].iloc[0]
nn = df.query("stop_id_original == 'HICHICNN'")['nearest_node'].iloc[0]
cn, nn

(1443059058, 3768324372)

In [7]:
df.query("stop_id_original == 'MCC5_1'")

,stop_sequence,stop_id_original,stop_name,map_latitude,map_longitude,nearest_node,nearest_edge
1063,1,MCC5_1,CENTRAL 5TH AVE - BAY 1,36.167091,-86.781923,1443059058,"(202338737, 202338859, 0)"


In [8]:
df.query("stop_id_original == 'HICHICNN'")

,stop_sequence,stop_id_original,stop_name,map_latitude,map_longitude,nearest_node,nearest_edge
1933,1,HICHICNN,HICKORY HOLLOW PL & HICKORY HOLLOW,36.052637,-86.654878,3768324372,"(2474131189, 3768324372, 0)"


## Travel time and distances for all pairs of stops
* uses GTFS instead of OSM (but )

In [2]:
fp = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/stops_node_matching.pkl'
stops_nodes_df = pd.read_pickle(fp)

fp = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/davidson_graph.graphml'
G = ox.load_graphml(fp)

In [7]:
stops_nodes_df.head()

,stop_sequence,stop_id_original,stop_name,map_latitude,map_longitude,nearest_node,nearest_edge
0,24,12ADEMNN,12TH AVE S & DEMONBREUN ST NB,36.154509,-86.786056,202342839,"(202284281, 202342839, 0)"
1,22,12ADIVNN,12TH AVE S & DIVISION ST NB,36.150417,-86.784338,202189411,"(202315471, 202300128, 0)"
2,24,16SFATSN,16TH ST & FATHERLAND ST SB,36.173512,-86.742468,202301462,"(202301462, 202324500, 0)"
3,21,19SFATNN,S 19TH ST & FATHERLAND ST NB,36.172884,-86.737058,202301467,"(202301467, 202310609, 0)"
4,47,1SOLDSM,N 1ST ST & OLDHAM ST SB,36.172806,-86.774050,202374591,"(202374591, 7251999645, 0)"


In [ ]:
# # MCC_17 as depot
# Occupancy as non-zero value, and use regression LSTM
# initialization condition at the start of the trip

In [30]:
import itertools

res = []
combinations = itertools.combinations(stops_nodes_df.stop_id_original.tolist(), 2)
for cn, nn in combinations:
    cn_df = stops_nodes_df.query("stop_id_original == @cn").reset_index()
    nn_df = stops_nodes_df.query("stop_id_original == @nn").reset_index()
    nn_df.columns = [f"{col}_nn" for col in nn_df.columns]
    df = cn_df.join(nn_df)
    res.append(df)
    # break
res = pd.concat(res)
res